In [ ]:
import matplotlib.pyplot as plt
import os
os.chdir("/path/to/work_dir")

# plt.style.use('ggplot')
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
inline_rc = dict(mpl.rcParams)
mpl.rcParams.update(mpl.rcParamsDefault)
import numpy as np
import re
from natsort import natsorted
def read_result(path): 
    preds = []
    labels = []
    with open(f'{path}.csv','r') as f:
        results = f.readlines()
        for idx, r in enumerate(results):
            r = r.strip()
            text = r.split('\t')
            label = 1 if text[2] == 'entailment' else 0
            pred = 1 if text[3] == 'entailment' else 0
            label = -1 if text[2] == 'discard' else label
            pred = -1 if text[3] == 'discard' else pred
            labels.append(label)
            preds.append(pred)
    return labels, preds
import numpy as np

def partition(lst, gap):
    steps = int(len(lst)/gap)
    lst_d = []
    for i in range(steps):
        lst_d.append(lst[i*gap:(i+1)*gap])
    return lst_d
model_name = "bert-large-cased"
dimension = ['length','price','mass','temperature']

pos_pred_list_all = []
neg_pred_list_all = []

for D in dimension:
    result_path = f"result/ASP_model/{model_name}-{D}-ei/"
    file_list = os.listdir(result_path)

    print(result_path)   
    file_list = natsorted(file_list)
    file_list = ["ordering","dimension","argument",
                "booster","diminisher","negation",
                "comparative","superlative"]
    labels = []
    preds = []
    nums = np.arange(60,140,2)
    print(nums.shape)
    pred_list = []
    label_list = []
    for file_name in file_list:
        file_path = f"{result_path}/{file_name}"
        labels, preds = read_result(file_path)
        pred_list.append(preds)
        label_list.append(labels)


    pos_pred_list = []
    neg_pred_list = []

    pos_label_list = []
    neg_label_list = []
    pair_num = []

    for preds, labels in zip(pred_list,label_list):
        label_div = partition(labels,40)
        pred_div = partition(preds,40)
        pair_num.append(len(label_div)/2)
        pos_labels = np.array(label_div[0])
        neg_labels = np.array(label_div[1])

        pos_preds = np.array([0]*40)
        neg_preds = np.array([0]*40)

        for idx,lst in enumerate(label_div):
            if idx % 2==0:
                pos_preds += pred_div[idx]
            else:
                neg_preds += pred_div[idx]



        pos_pred_list.append(pos_preds)
        neg_pred_list.append(neg_preds)
        pos_label_list.append(pos_labels)
        neg_label_list.append(neg_labels)

    len(pos_pred_list)
    pos_pred_list_all.append(pos_pred_list)
    neg_pred_list_all.append(neg_pred_list)

pos_pred_list_all = np.array(pos_pred_list_all)
pos_pred_list = np.sum(pos_pred_list_all,axis=0).tolist()
neg_pred_list_all = np.array(neg_pred_list_all)
neg_pred_list = np.sum(neg_pred_list_all,axis=0).tolist()
print(pos_pred_list)


In [ ]:

from ASP_utils import *
from sklearn import preprocessing
import numpy as np
import random
import sklearn.metrics
def possigmoid(x, a, b):
    return 1.0 / (1.0 + np.exp(-a*(x-b)))
def negsigmoid(x, a, b):
    return 1 - (1.0 / (1.0 + np.exp(-a*(x-b))))
from scipy.optimize import curve_fit

min_max_scaler = preprocessing.MinMaxScaler()
test_type = ["Ordering","Dimension","Argument",
            "Booster","Diminisher","Negation",
            "Comparative","Superlative"]
human_label = ["Ordering",'Booster','Diminisher',
            'Negation','Comparative','Superlative']

print(model_name)
pos_pred_list_p = []
neg_pred_list_p = []

pos_label_list_p = []
neg_label_list_p = []
metrics = 'accuracy'
for t,n_pair,pos_p,pos_l,neg_p,neg_l in zip(test_type,pair_num,
                                            pos_pred_list,pos_label_list,
                                            neg_pred_list,neg_label_list):
    n_pair = n_pair *len(dimension)
    pos_p = np.array(pos_p)
    pos_l = np.array(pos_l)
    neg_p = np.array(neg_p)
    neg_l = np.array(neg_l)

    pos_pred_list_p.append(np.array(pos_p)/n_pair)
    neg_pred_list_p.append(np.array(neg_p)/n_pair)
    if t in human_label:

        pos_label = read_pkl(f"/human/pkl/{t.lower()}_pos.pkl")[60:140]
        neg_label = read_pkl(f"/human/pkl/{t.lower()}_neg.pkl")[60:140]
        #print(len(pos_preds))
        pos_label = pos_label[::2]
        neg_label = neg_label[::2]

        pos_label = min_max_scaler.fit_transform(pos_label)
        pos_label = pos_label.squeeze()
        neg_label = min_max_scaler.fit_transform(neg_label)
        neg_label = neg_label.squeeze()
        varible = True
    else:
        pos_label = pos_l
        neg_label = neg_l
        varible = False
    pos_baseline0 = np.array(np.where(pos_p>=0,0,-1))
    pos_baseline1 = np.array(np.where(pos_p>=0,1,-1))

    neg_baseline0 = np.array(np.where(neg_p>=0,0,-1))
    neg_baseline1 = np.array(np.where(neg_p>=0,1,-1))

    sim_baseline =0
    sim_baseline += max(compute_metrics(pos_baseline0,pos_label,metrics,n_pair,varible,'pos'),
                        compute_metrics(pos_baseline1,pos_label,metrics,n_pair,varible,'pos'))

    sim_baseline += max(compute_metrics(neg_baseline0,neg_label,metrics,n_pair,varible,'neg'),
                        compute_metrics(neg_baseline1,neg_label,metrics,n_pair,varible,'neg'))
    pos_label_list_p.append(pos_label)
    neg_label_list_p.append(neg_label)
    pos_p = np.array(pos_p)
    neg_p = np.array(neg_p)

    sim = compute_metrics(np.array(pos_p),pos_label,metrics,n_pair,varible,'pos',is_weighted=False)
    #print(sim)
    sim += compute_metrics(np.array(neg_p),neg_label,metrics,n_pair,varible,'neg',is_weighted=False)
    print(sim/2)

In [ ]:
test_type = ["ordering","dimension","argument",
            "booster","diminisher","negation",
            "comparative","individual","superlative"]

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']
plt.rcParams['font.size'] = 15
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['savefig.dpi'] = 300 
plt.rcParams['figure.dpi'] = 300 
plt.rc('font',family='Times New Roman')
plt.figure(figsize=(4.17,2.4))

std_color = ['#1f77b4', '#ff7f0e', '#2ca02c', 
            '#d62728', '#9467bd', '#8c564b', 
            '#e377c2', '#7f7f7f', '#bcbd22', 
            '#17becf']
light_color = ['#6aa5ce', '#ffaa5f', '#73c073', 
            '#e47070', '#b89ad4', '#b38f88', 
            '#eda5d7', '#aaaaaa', '#d3d46c', 
            '#65d4e0']
marker_type = []
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', 
            '#d62728', '#9467bd', '#8c564b', 
            '#e377c2', '#7f7f7f', '#bcbd22', 
            '#17becf']
# colors = ['#1F2B50','#5A212D','#326623',]
colors = ['grey', '#ff7f0e', '#2ca02c', 
            '#d62728', '#9467bd', '#1f77b4', 
            '#e377c2', '#7f7f7f', '#bcbd22', 
            '#17becf']
line_type = ['solid',(0,(4,1)),'dotted']
for idx, pos_preds in enumerate(pos_pred_list_p):
    idx = len(pos_pred_list_p) - idx - 1
    pos_preds = pos_pred_list_p[idx]
    if idx in [1,2,6,7]:
        continue

    lw = 2.5 if idx == 0 else 2.0
    zorder =1 if idx == 0 else 3
    plt.plot(nums,pos_preds,marker=None,color=colors[idx],label=test_type[idx],
    lw=lw,ls=line_type[idx//3],alpha=0.9,zorder=zorder)

    plt.xlim([60,138])
    plt.ylim([-0.05,1.05])
    plt.ylabel('  ')
plt.xticks(np.arange(60,140,10),['0.6','0.7','0.8','0.9','1.0','1.1','1.2','1.3'])
plt.yticks([0.0,0.5,1.0],['0.0','0.5','1.0'])
plt.xticks(np.arange(60,140,20),['0.6','0.8','1.0','1.2'])
plt.xticks(np.arange(60,140,20),['60','80','100','120'])
plt.xticks([60,80,100,120,138],['60','80','100','120','138'])


plt.xlabel("    ")
plt.tight_layout()
plt.savefig(f'plot/img/{model_name}_pos.png',bbox_inches='tight',pad_inches=0.0)

plt.show()
plt.figure(figsize=(4.17,2.4))

for idx, neg_preds in enumerate(neg_pred_list_p):
    
    idx = len(pos_pred_list_p) - idx - 1
    neg_preds = neg_pred_list_p[idx]
    lw = 2.5 if idx == 0 else 2.0
    zorder =1 if idx == 0 else 3

    if idx in [1,2,6,7]:
        continue
    plt.plot(nums,neg_preds,marker=None,color=colors[idx],label=test_type[idx],
    lw=lw,ls=line_type[idx//3],alpha=0.9,zorder=zorder)

    plt.xlim([60,138])
    plt.ylim([-0.05,1.05])
    plt.ylabel('    ')

plt.xticks(np.arange(60,140,10),['0.6','0.7','0.8','0.9','1.0','1.1','1.2','1.3'])
plt.xticks(np.arange(60,140,20),['0.6','0.8','1.0','1.2'])
plt.xticks(np.arange(60,140,20),['60','80','100','120'])
plt.xticks([60,80,100,120,138],['60','80','100','120','138'])


plt.xlabel("    ")
plt.yticks([0.0,0.5,1.0],['0.0','0.5','1.0'])

plt.tight_layout()
plt.savefig(f'plot/img/{model_name}_neg.png',bbox_inches='tight',pad_inches=0.0)
plt.show()